# ENC - master Humanités numériques

# Évaluation pour le cours d'apprentissage machine : question 2

## Question : Random forest sur MNIST
Proposez un code permettant d'effectuer des prédictions des catégories de MNIST en utilisant un algorithme de random forest.\
Evaluez la performance avec une cross-validation.

## Mise en oeuvre

### On importe les différents modules et le jeu de données

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

import numpy as np
import pandas as pd

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()


11493376/11490434 [==============================] - 0s 0us/step


In [0]:
print(f"x_train : {x_train.shape}")
print(f"x_test : {x_test.shape}")
print(f"y_train : {y_train.shape}")
print(f"y_test : {y_test.shape}")

x_train : (60000, 28, 28)
x_test : (10000, 28, 28)
y_train : (60000,)
y_test : (10000,)


On transforme les jeux d'images en tableaux.

In [0]:
flat_x_train = x_train.reshape(60000,784)
flat_x_test = x_test.reshape(10000,784)

On normalise la valeur des pixels.

In [0]:
flat_x_train = flat_x_train / 255
flat_x_test = flat_x_test / 255

### Modèle

#### On crée le modèle

In [0]:
rfc = RandomForestClassifier(n_estimators=200, random_state=5)

#### On entraîne le modèle

In [0]:
rfc.fit(flat_x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=5, verbose=0,
                       warm_start=False)

#### On évalue le modèle sur les données de test.



In [0]:
print(f"Le modèle obtient un taux de validité des prédictions de {rfc.score(flat_x_test, y_test)}.")

Le modèle obtient un taux de validité des prédictions de 0.9705.


#### On évalue ensuite le modèle par validation croisée.

Le programme va générer lui-même les données d'entraînement et de test, on peut donc recréer un jeu unique.

In [0]:
flat_x = np.concatenate([flat_x_train, flat_x_test], axis=0)
print(flat_x.shape)
y = np.concatenate([y_train, y_test], axis=0)
print(y.shape)

(70000, 784)
(70000,)


On lance une validation croisée (*k-fold cross validation*) à 5 plis (i.e. le programme va partager le jeu en 5 parties).

In [0]:
rfc_cv_score = cross_val_score(rfc, flat_x, y, cv=5)

Pour chacune des 5 parties, on obtient les taux suivant de validité des prédictions :

In [0]:
print(rfc_cv_score)

[0.96957143 0.96871429 0.96664286 0.96635714 0.97292857]


On calcule la moyenne et la variance des 5 scores :

In [0]:
print(f"Score moyen : {rfc_cv_score.mean()}, variance : {rfc_cv_score.std() * 2}")

Score moyen : 0.968842857142857, variance : 0.0047547183869053695


**Interprétation**\
Le modèle obtient un taux d'exactitude moyen de 96,88 %.\
L'écart entre chacun des scores obtenus est très faible, le modèle est très peu dépendant des parties du jeu de données.